## Verification of Model Forecast vs Observations (Measurements)

More information can be found here:
- https://opendatadocs.dmi.govcloud.dk/APIs/Meteorological_Observation_API
- https://scores.readthedocs.io/en/stable/index.html

In [ ]:
path_model = "/users/sadamov/pyprojects/neural-lam/data/danra/single_levels.zarr"
url_model = "https://mllam-test-data.s3.eu-north-1.amazonaws.com/single_levels.zarr"
vars_model = ["u10", "v10"]

path_obs = "/users/sadamov/pyprojects/neural-lam/data/danra/observations.zarr"
url_obs = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"
vars_obs = ["wind_speed_past1h", "wind_dir_past1h"]
api_key = "8dff599e-9a48-46eb-a166-72f2f722645e"

In [ ]:
import requests
import os
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd

In [ ]:
if not os.path.exists(path_model):
    ds_model = xr.open_zarr(url_model)
    chunk_dict = {dim: -1 for dim in ds_model.dims if dim != "time"}
    chunk_dict["time"] = 20
    ds_model = ds_model.chunk(chunk_dict)

    for var in ds_model.variables:
        if "chunks" in ds_model[var].encoding:
            del ds_model[var].encoding["chunks"]

    ds_model.to_zarr(path_model, mode="w")
else:
    ds_model = xr.open_zarr(path_model)

In [ ]:
datetime_start = np.datetime_as_string(ds_model.time.values[0], unit="s")
datetime_end = np.datetime_as_string(ds_model.time.values[-1], unit="s")
datetime_range = f"{datetime_start}Z/{datetime_end}Z"
datetime_range = "2020-09-01T00:00:00Z/2020-09-13T09:00:00Z"
print(datetime_range)

In [ ]:
# Initialize an empty dictionary to store DataFrames for each variable
dfs = {}

# Loop over each variable
for var in vars_obs:
    # Adjust the params for the current variable
    params = {
        "api-key": api_key,
        "datetime": datetime_range,
        "parameterId": var,
    }

    # Fetch the data
    response = requests.get(url_obs, params=params)
    data = response.json()

    # Assuming 'data' is your GeoJSON object
    gdf = gpd.GeoDataFrame.from_features(data["features"])

    # Convert 'observed' to datetime in UTC
    gdf["time"] = pd.to_datetime(gdf["observed"], utc=True)

    # Pivot the DataFrame
    df_pivot = gdf.pivot(index="time", columns="stationId", values="value")

    # Store the pivoted DataFrame in the dictionary
    dfs[var] = df_pivot

# Get the time index from the first DataFrame and convert to numpy.datetime64
time_index = dfs[vars_obs[0]].index.tz_convert(
    None).to_numpy().astype('datetime64[ns]')

# Combine the DataFrames into a single xarray Dataset
ds_obs = xr.Dataset(
    {var: (["time", "stationId"], dfs[var].values) for var in vars_obs},
    coords={
        "time": time_index,
        "stationId": dfs[vars_obs[0]].columns,
        "lat": ("stationId", gdf.groupby("stationId")["geometry"].first().y),
        "lon": ("stationId", gdf.groupby("stationId")["geometry"].first().x),
    }
)

# Sort by time
ds_obs = ds_obs.sortby("time")
ds_obs

In [ ]:
ds_obs["u10"] = ds_obs["wind_speed_past1h"] * np.cos(
    np.radians(90 - ds_obs["wind_dir_past1h"])
)
ds_obs["v10"] = ds_obs["wind_speed_past1h"] * np.sin(
    np.radians(90 - ds_obs["wind_dir_past1h"])
)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a new figure with a specific projection
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot the scatter plot
scatter = ax.scatter(ds_obs["lon"], ds_obs["lat"], c=ds_obs.isel(time=0)["u10"], cmap="coolwarm", transform=ccrs.PlateCarree())

# Add coastlines
ax.add_feature(cfeature.COASTLINE)

# Add borders
ax.add_feature(cfeature.BORDERS)

# Add gridlines
ax.gridlines(draw_labels=True)

# Add colorbar
plt.colorbar(scatter, label='u10')

# Set the extent of the map (adjust these values based on your data)
ax.set_extent([ds_obs["lon"].min(), ds_obs["lon"].max(), ds_obs["lat"].min(), ds_obs["lat"].max()])

# Show the plot
plt.title('U10 Wind Component')
plt.show()